In [3]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize_scalar
from datetime import time

# Project research log
### NOTHING has been produced using the help of any AI Assistant, all the code and the analysis of the results is the fruit of manual work.

Data pipeline

In [4]:
data = pd.read_csv('big_dataset.csv')

/var/folders/qj/v9ln2xk93vjb08vw8zv44f8r0000gn/T/ipykernel_41159/244950733.py:1: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('big_dataset.csv')


In [5]:
data

,DATE,TIME_M,EX,SYM_ROOT,SYM_SUFFIX,BID,BIDSIZ,ASK,ASKSIZ,QU_COND,BIDEX,ASKEX,QU_SEQNUM,NATBBO_IND,NASDBBO_IND,QU_CANCEL,QU_SOURCE
0,2010-01-04,9:30:00.020,T,AAPL,NaN,213.40,1,213.50,1,R,T,T,206749,NaN,0,NaN,N
1,2010-01-04,9:30:00.020,T,AAPL,NaN,213.40,2,213.50,1,R,T,T,206754,NaN,2,NaN,N
2,2010-01-04,9:30:00.023,T,AAPL,NaN,213.40,1,213.50,1,R,T,T,206772,NaN,2,NaN,N
3,2010-01-04,9:30:00.023,T,AAPL,NaN,213.41,1,213.50,1,R,T,T,206773,NaN,2,NaN,N
4,2010-01-04,9:30:00.023,T,AAPL,NaN,213.42,1,213.50,1,R,T,T,206778,NaN,2,NaN,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15712981,2010-01-08,16:00:55.271,I,XLF,NaN,15.21,22,15.25,20,R,I,I,6634637,0.0,2,A,C
15712982,2010-01-08,16:00:55.274,I,XLF,NaN,15.21,22,15.26,314,R,I,I,6634638,0.0,2,A,C
15712983,2010-01-08,16:00:55.704,T,XLF,NaN,15.21,63,15.23,74,R,T,T,6634639,6.0,2,A,C
15712984,2010-01-08,16:00:57.960,T,XLF,NaN,15.21,90,15.23,74,R,T,T,6634652,0.0,2,A,C


In [6]:
# We do all the data cleaning described in the article
"""
- subset of exchanges
- time columns construction and sorting 
- subset of data 10:00:00am < data < 4:00:00 floor at the seconde
- No zero BID / ASK and <0 spread 
- We don't delete mode != 12 as it not explicitly mentioned in the article    
"""


data = data[data.EX.isin(['T', 'P', 'Z'])]
data["ts"] = pd.to_datetime(data["DATE"].astype(str) + " " + data["TIME_M"], utc=True, errors="coerce")
data["ts"] = data['ts'].dt.floor('s')
data = data.sort_values(["SYM_ROOT","EX","ts"])
data = data[(data.ts.dt.time >= time(10, 00, 0)) & (data.ts.dt.time <= time(15, 59, 59))]
data = data[(data.ASK > 0) & (data.BID > 0)]
data['spread'] = data['ASK'] - data['BID']
data = data[data['spread'] > 0]
# data = data[data.MODE == 12] # regular quotes 
data["spread_cents"] = np.round((data["spread"]) * 100).astype("Int64")

/var/folders/qj/v9ln2xk93vjb08vw8zv44f8r0000gn/T/ipykernel_41159/3222834321.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["ts"] = pd.to_datetime(data["DATE"].astype(str) + " " + data["TIME_M"], utc=True, errors="coerce")
/var/folders/qj/v9ln2xk93vjb08vw8zv44f8r0000gn/T/ipykernel_41159/3222834321.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["ts"] = data['ts'].dt.floor('s')


In [7]:
data = data.rename(columns={'SYM_ROOT':'SYMBOL', 'ASK':'OFR', 'ASKSIZ':'OFRSIZ', 'TIME_M':'TIME'})[['DATE','TIME','EX','SYMBOL','BID','OFR','OFRSIZ','BIDSIZ','spread', 'spread_cents']]
data['TIME'] = data['TIME'].str[:-4]
data

,DATE,TIME,EX,SYMBOL,BID,OFR,OFRSIZ,BIDSIZ,spread,spread_cents
125010,2010-01-04,10:00:00,P,AAPL,213.25,213.37,8,2,0.12,12
125011,2010-01-04,10:00:00,P,AAPL,213.37,213.38,4,1,0.01,1
125012,2010-01-04,10:00:00,P,AAPL,213.37,213.47,2,1,0.10,10
125016,2010-01-04,10:00:00,P,AAPL,213.37,213.46,1,1,0.09,9
125017,2010-01-04,10:00:00,P,AAPL,213.36,213.46,1,1,0.10,10
...,...,...,...,...,...,...,...,...,...,...
15712226,2010-01-08,15:59:59,Z,XLF,15.22,15.23,630,310,0.01,1
15712228,2010-01-08,15:59:59,Z,XLF,15.22,15.23,330,310,0.01,1
15712239,2010-01-08,15:59:59,Z,XLF,15.22,15.23,280,310,0.01,1
15712243,2010-01-08,15:59:59,Z,XLF,15.22,15.23,330,310,0.01,1


In [12]:
# QQQQ is not a valid ticker on WRDS -> we merge we provided data

QQQQ_data = pd.read_csv('data-2.csv')
QQQQ_data = QQQQ_data[QQQQ_data.SYM_ROOT == 'QQQQ']
QQQQ_data = QQQQ_data[QQQQ_data.EX.isin(['T', 'P', 'Z'])]
QQQQ_data["ts"] = pd.to_datetime(QQQQ_data["DATE"].astype(str) + " " + QQQQ_data["TIME_M"], utc=True, errors="coerce")
QQQQ_data["ts"] = QQQQ_data['ts'].dt.floor('s')
QQQQ_data = QQQQ_data.sort_values(["SYM_ROOT","EX","ts"])
QQQQ_data = QQQQ_data[(QQQQ_data.ts.dt.time >= time(10, 00, 0)) & (QQQQ_data.ts.dt.time <= time(15, 59, 59))]
QQQQ_data = QQQQ_data[(QQQQ_data.ASK > 0) & (QQQQ_data.BID > 0)]
QQQQ_data['spread'] = QQQQ_data['ASK'] - QQQQ_data['BID']
QQQQ_data = QQQQ_data[QQQQ_data['spread'] > 0]
# data = data[data.MODE == 12] # regular quotes
QQQQ_data["spread_cents"] = np.round((QQQQ_data["spread"]) * 100).astype("Int64")
QQQQ_data = QQQQ_data.rename(columns={'SYM_ROOT':'SYMBOL', 'ASK':'OFR', 'ASKSIZ':'OFRSIZ', 'TIME_M':'TIME'})[['DATE','TIME','EX','SYMBOL','BID','OFR','OFRSIZ','BIDSIZ','spread', 'spread_cents']]
QQQQ_data['TIME'] = QQQQ_data['TIME'].str[:-4]
QQQQ_data

/var/folders/qj/v9ln2xk93vjb08vw8zv44f8r0000gn/T/ipykernel_41159/423266644.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  QQQQ_data = pd.read_csv('data-2.csv')


,DATE,TIME,EX,SYMBOL,BID,OFR,OFRSIZ,BIDSIZ,spread,spread_cents
8377955,20100104,10:00:00,P,QQQQ,46.34,46.36,60,72,0.02,2
8378006,20100104,10:00:00,P,QQQQ,46.35,46.37,42,7,0.02,2
8378008,20100104,10:00:00,P,QQQQ,46.35,46.37,48,52,0.02,2
8378013,20100104,10:00:00,P,QQQQ,46.35,46.37,48,52,0.02,2
8378014,20100104,10:00:00,P,QQQQ,46.36,46.37,42,6,0.01,1
...,...,...,...,...,...,...,...,...,...,...
18328118,20100108,15:59:59,Z,QQQQ,46.54,46.55,1282,944,0.01,1
18328119,20100108,15:59:59,Z,QQQQ,46.54,46.55,1287,944,0.01,1
18328121,20100108,15:59:59,Z,QQQQ,46.54,46.55,1292,944,0.01,1
18328129,20100108,15:59:59,Z,QQQQ,46.54,46.55,1292,930,0.01,1


In [13]:
data = pd.concat([data, QQQQ_data])

In [42]:
# data = pd.read_csv('wdrs_data.csv')
# data["ts"] = pd.to_datetime(data["DATE"].astype(str) + " " + data["TIME"], utc=True, errors="coerce")
# data = data.sort_values(["SYMBOL","EX","ts"])
# data = data[(data.ts.dt.time >= time(10, 00, 0)) & (data.ts.dt.time <= time(16, 0, 0))]
# data = data[(data.OFR > 0) & (data.BID > 0)]
# data['spread'] = data['OFR'] - data['BID']
# data = data[data['spread'] > 0]
# data = data[data.MODE == 12] # regular quotes
# data["spread_cents"] = np.round((data["spread"]) * 100).astype("Int64")
# display(data)

In [9]:
# We also study MSFT 

TICKERS = ['AAPL','XLF','QQQQ','JPM','MSFT']
EX = ['T','P','Z']
CANDLES = (6*3600) * 5 # 5 days -> 6 hours + 16:00:00 -> 16:00:59

Empirical estimation

In [14]:
def get_stats(ticker, ex):
    res = {'Ticker':ticker, "Exchange": ex}
    sub = data[(data['SYMBOL'] == ticker) & (data['EX'] == ex)].copy() # By ticker and Ex

    res['num quotes'] = np.round(sub.shape[0]/1_000_000,1) # len 
    res['quotes/sec'] = np.round(sub.groupby(['DATE','TIME']).size().sum()/CANDLES,0) # count every second even if there is no quote
    res['avg spread'] = np.round(sub['spread'].mean(),3) 
    res['avg(bsize + asize)'] = np.round(sub['BIDSIZ'].mean()+sub['OFRSIZ'].mean(),1) 
    res['avg(price)'] = np.round((sub['BID'].mean()+sub['OFR'].mean())/2,2)

    return res

In [15]:
# Table 2 as in the article
table2 = [get_stats(ticker, ex) for ticker in TICKERS for ex in EX]
table2 = pd.DataFrame(table2)
table2 = table2.groupby(['Ticker', 'Exchange']).first()
display(table2)

num quotes  quotes/sec  avg spread  avg(bsize + asize)  \
Ticker Exchange                                                           
AAPL   P                0.4         4.0       0.046                 5.7   
       T                1.4        13.0       0.035                 9.2   
       Z                0.7         6.0       0.054                 4.5   
JPM    P                0.8         7.0       0.067                41.8   
       T                1.2        11.0       0.341                84.5   
       Z                0.6         5.0       0.016                38.8   
MSFT   P                0.4         4.0       0.011               233.5   
       T                0.9         9.0       0.011               506.1   
       Z                0.5         5.0       0.013               221.3   
QQQQ   P                4.0        37.0       0.011              1151.9   
       T                2.7        25.0       0.010              1455.2   
       Z                1.6        15.0       0.011              1055.4   
XLF    P                0.4         4.0       0.010             10466.8   
       T                0.7         7.0       0.010              8799.5   
       Z                0.4         4.0       0.011              7507.8   

                 avg(price)  
Ticker Exchange              
AAPL   P             212.66  
       T             212.50  
       Z             212.43  
JPM    P              41.34  
       T              43.46  
       Z              43.77  
MSFT   P              30.74  
       T              30.73  
       Z              30.73  
QQQQ   P              46.27  
       T              46.30  
       Z              46.28  
XLF    P              15.01  
       T              15.02  
       Z              15.00

> Despite performing the same filtering as described in the artcile, we are not able to retreive perfect match with the TABLE 2 provided in the article \
> This must resort either from a difference in the orignal datset or a non discose filtering (we have slightly more data than in the article after filtering)

Model fit and validation

In [16]:
NasDaq = data[data.EX == 'T'].copy()
NasDaq.loc[(NasDaq.spread_cents == 1) & (NasDaq.SYMBOL == 'AAPL'), "SYMBOL"] = "AAPL1"
NasDaq.loc[(NasDaq.spread_cents == 2) & ((NasDaq.SYMBOL == 'AAPL')), "SYMBOL"] = "AAPL2"
NasDaq.loc[(NasDaq.spread_cents >= 3) & ((NasDaq.SYMBOL == 'AAPL')), "SYMBOL"] = "AAPL3"

In [17]:
def bucket_deciles(df, cols=("BIDSIZ","OFRSIZ"), by=None, suffix="_dec"):
    for c in cols:
        # We choose the first bucket value -> closest to the artcile results
        r = df.groupby(by)[c].rank(pct=True, method="first") if by else df[c].rank(pct=True, method="first")
        d = pd.cut(r, np.linspace(0,1,11), labels=range(1,11), include_lowest=True).astype("Int64")
        d[df[c].isna()] = pd.NA
        df[f"{c}{suffix}"] = d
    return df

NasDaq = bucket_deciles(NasDaq, ["BIDSIZ","OFRSIZ"], by=["SYMBOL"])

In [18]:
NasDaq['price'] =(NasDaq['BID']+NasDaq['OFR'])/2

In [19]:
# Quick check on the deciles

def deciles_table(df, cols=("BIDSIZ","OFRSIZ")):
    qs = np.arange(0.1, 1.0, 0.1)  # 0.1, 0.2, ..., 0.9
    out = {}
    for c in cols:
        q = df[c].dropna().quantile(qs)
        out[c] = np.round(q.values, 0).astype(int)
    idx = [f"p{int(100*q)}" for q in qs]
    return pd.DataFrame(out, index=idx)

def print_deciles_for_ticker(df, ticker):
    sub = df[df["SYMBOL"] == ticker]
    tab = deciles_table(sub, cols=("BIDSIZ","OFRSIZ"))
    print(f"\nDéciles p10..p90 pour {ticker} @ Nasdaq (T) — BIDSIZ / OFRSIZ")
    print(tab)
    return tab

# test on XLF
tab_xlf = print_deciles_for_ticker(NasDaq, "XLF")


Déciles p10..p90 pour XLF @ Nasdaq (T) — BIDSIZ / OFRSIZ
     BIDSIZ  OFRSIZ
p10    1436    1112
p20    2277    1808
p30    3351    2508
p40    4505    3445
p50    5237    4541
p60    5623    5279
p70    5985    5770
p80    6427    6125
p90    6995    6669


In [20]:
def compute_u_ij_emp(ticker):
    sub = NasDaq[NasDaq.SYMBOL == ticker]
    chg = np.sign(sub['price'].diff().to_numpy())  # change in price  {-1, 0, +1}
    n = len(chg)
    pos = np.flatnonzero(chg != 0) #Indices where the mid-price actually changes (non-zero moves)
    k = np.searchsorted(pos, np.arange(n), side="right")  # For each row i, find the index of the next price-change strictly after i
    valid = k < len(pos)
    next_sign = np.full(n, np.nan) # Next-change sign for each row (NaN if no future change)
    next_sign[valid] = chg[pos[k[valid]]]
    up_next = (next_sign > 0).astype(float) #does the next change go up? (1.0 yes, 0.0 no)
    m = pd.DataFrame({
        'bid_size_dec': sub['BIDSIZ_dec'].to_numpy(),
        'ask_size_dec': sub['OFRSIZ_dec'].to_numpy(),
        'up_next': up_next # up indicator
    }).dropna(subset=['bid_size_dec', 'ask_size_dec', 'up_next'])

    #For each bucket (i = bid decile, j = ask decile), compute the empirical
    #probability that the next mid-price change is up: u_ij = mean(up_next)
    #returned as a matrix with rows = bid decile (i) and columns = ask decile (j)
    uij = (m.groupby(['ask_size_dec', 'bid_size_dec'])['up_next']
             .mean()
             .unstack('ask_size_dec')
             .sort_index()).fillna(0)
    return uij

In [21]:
p_XLF = compute_u_ij_emp('XLF')
p_QQQQ = compute_u_ij_emp('QQQQ')
p_AAPL1 = compute_u_ij_emp('AAPL1')
p_AAPL2 = compute_u_ij_emp('AAPL2')
p_AAPL3 = compute_u_ij_emp('AAPL3')
p_JPM = compute_u_ij_emp('JPM')
p_MSFT = compute_u_ij_emp('MSFT')

In [22]:
display(np.round(p_XLF,3))

ask_size_dec,1,2,3,4,5,6,7,8,9,10
bid_size_dec,,,,,,,,,,
1,0.631,0.593,0.153,0.283,0.558,0.364,0.301,0.206,0.266,0.057
2,0.688,0.321,0.391,0.913,0.308,0.415,0.361,0.319,0.267,0.218
3,0.670,0.731,0.944,0.785,0.384,0.449,0.457,0.472,0.463,0.404
4,0.936,0.819,0.545,0.623,0.463,0.521,0.482,0.516,0.520,0.396
5,0.662,0.562,0.570,0.644,0.564,0.554,0.627,0.555,0.561,0.552
6,0.794,0.350,0.692,0.586,0.550,0.536,0.506,0.498,0.587,0.493
7,0.889,0.724,0.595,0.535,0.611,0.615,0.591,0.638,0.557,0.593
8,0.913,0.787,0.720,0.702,0.667,0.601,0.562,0.656,0.671,0.534
9,0.987,0.914,0.814,0.722,0.632,0.694,0.688,0.744,0.810,0.649


In [23]:
display(np.round(p_QQQQ,3))

ask_size_dec,1,2,3,4,5,6,7,8,9,10
bid_size_dec,,,,,,,,,,
1,0.462,0.265,0.271,0.271,0.275,0.243,0.174,0.230,0.202,0.256
2,0.644,0.505,0.497,0.387,0.359,0.316,0.405,0.295,0.293,0.259
3,0.694,0.551,0.531,0.496,0.406,0.442,0.418,0.363,0.345,0.509
4,0.705,0.636,0.555,0.508,0.475,0.535,0.315,0.448,0.397,0.333
5,0.732,0.634,0.580,0.526,0.404,0.433,0.423,0.426,0.476,0.207
6,0.823,0.709,0.670,0.662,0.560,0.518,0.453,0.456,0.506,0.662
7,0.801,0.686,0.644,0.640,0.700,0.616,0.545,0.527,0.503,0.325
8,0.666,0.669,0.686,0.629,0.776,0.682,0.572,0.509,0.473,0.556
9,0.744,0.702,0.681,0.604,0.609,0.399,0.594,0.233,0.391,0.481


In [24]:
display(np.round(p_AAPL1,3))

ask_size_dec,1,2,3,4,5,6,7,8,9,10
bid_size_dec,,,,,,,,,,
1,0.501,0.000,0.472,0.440,0.450,0.444,0.410,0.423,0.414,0.438
2,0.536,0.510,0.000,0.483,0.509,0.424,0.424,0.425,0.412,0.412
3,0.511,0.550,0.484,0.585,0.456,0.465,0.439,0.412,0.416,0.421
4,0.542,0.528,0.544,0.494,0.455,0.455,0.452,0.444,0.422,0.386
5,0.561,0.658,0.541,0.507,0.508,0.474,0.479,0.479,0.470,0.398
6,0.602,0.563,0.552,0.530,0.517,0.500,0.488,0.481,0.449,0.428
7,0.559,0.557,0.567,0.547,0.536,0.520,0.502,0.491,0.462,0.429
8,0.576,0.550,0.600,0.539,0.532,0.531,0.500,0.492,0.502,0.425
9,0.598,0.615,0.585,0.553,0.520,0.538,0.539,0.517,0.527,0.451


In [25]:
display(np.round(p_AAPL2,3))

ask_size_dec,1,2,3,4,5,6,7,8,9,10
bid_size_dec,,,,,,,,,,
1,0.504,0.533,0.449,0.466,0.399,0.399,0.384,0.342,0.312,0.297
2,0.527,0.523,0.435,0.478,0.390,0.443,0.394,0.332,0.350,0.331
3,0.510,0.556,0.497,0.478,0.456,0.444,0.419,0.388,0.389,0.354
4,0.710,0.564,0.573,0.513,0.440,0.478,0.449,0.427,0.393,0.401
5,0.584,0.588,0.531,0.538,0.493,0.479,0.456,0.427,0.425,0.378
6,0.658,0.593,0.569,0.541,0.487,0.515,0.523,0.464,0.444,0.425
7,0.646,0.625,0.603,0.574,0.539,0.551,0.510,0.483,0.451,0.429
8,0.658,0.660,0.592,0.605,0.526,0.562,0.509,0.470,0.473,0.438
9,0.646,0.682,0.616,0.628,0.564,0.575,0.506,0.491,0.498,0.436


In [26]:
display(np.round(p_AAPL3,3))

ask_size_dec,1,2,3,4,5,6,7,8,9,10
bid_size_dec,,,,,,,,,,
1,0.496,0.416,0.402,0.340,0.321,0.294,0.278,0.266,0.242,0.235
2,0.602,0.515,0.389,0.417,0.324,0.354,0.298,0.273,0.264,0.290
3,0.596,0.618,0.512,0.433,0.435,0.381,0.366,0.334,0.340,0.332
4,0.661,0.610,0.552,0.527,0.453,0.471,0.422,0.388,0.352,0.366
5,0.679,0.698,0.582,0.564,0.511,0.459,0.463,0.444,0.415,0.404
6,0.709,0.657,0.624,0.568,0.551,0.499,0.465,0.458,0.416,0.361
7,0.722,0.710,0.621,0.621,0.549,0.560,0.518,0.487,0.454,0.423
8,0.746,0.729,0.648,0.622,0.584,0.560,0.536,0.497,0.468,0.457
9,0.748,0.736,0.662,0.629,0.598,0.571,0.548,0.515,0.475,0.476


In [27]:
display(np.round(p_JPM,3))

ask_size_dec,1,2,3,4,5,6,7,8,9,10
bid_size_dec,,,,,,,,,,
1,0.680,0.392,0.318,0.275,0.212,0.178,0.137,0.109,0.084,0.064
2,0.601,0.459,0.390,0.364,0.293,0.257,0.232,0.179,0.143,0.115
3,0.623,0.497,0.420,0.375,0.335,0.310,0.265,0.223,0.198,0.173
4,0.628,0.483,0.424,0.391,0.361,0.328,0.291,0.245,0.227,0.174
5,0.668,0.495,0.443,0.409,0.378,0.346,0.312,0.260,0.239,0.173
6,0.677,0.515,0.460,0.397,0.380,0.374,0.341,0.297,0.260,0.179
7,0.688,0.544,0.475,0.431,0.405,0.371,0.362,0.302,0.261,0.207
8,0.725,0.559,0.502,0.448,0.428,0.411,0.380,0.300,0.263,0.219
9,0.757,0.607,0.543,0.511,0.456,0.458,0.413,0.340,0.292,0.267


In [79]:
display(np.round(p_MSFT,3))

ask_size_dec,1,2,3,4,5,6,7,8,9,10
bid_size_dec,,,,,,,,,,
1,0.495,0.387,0.402,0.293,0.211,0.191,0.132,0.118,0.128,0.073
2,0.673,0.559,0.490,0.387,0.342,0.304,0.298,0.232,0.221,0.196
3,0.681,0.575,0.498,0.449,0.391,0.367,0.394,0.326,0.343,0.274
4,0.755,0.589,0.563,0.518,0.500,0.432,0.410,0.399,0.347,0.321
5,0.822,0.655,0.550,0.529,0.568,0.514,0.473,0.391,0.390,0.360
6,0.804,0.623,0.563,0.568,0.577,0.574,0.500,0.506,0.407,0.286
7,0.859,0.726,0.644,0.623,0.576,0.553,0.516,0.504,0.483,0.289
8,0.881,0.736,0.691,0.678,0.595,0.521,0.501,0.496,0.466,0.302
9,0.875,0.763,0.752,0.667,0.620,0.533,0.573,0.562,0.502,0.410


In [28]:
def compute_dij(ticker):
    sub = NasDaq[NasDaq.SYMBOL == ticker]
    idx  = pd.Index(range(1, 11), name='BIDSIZ_dec') # Define the full 10×10 grid of decile labels for rows (bid) and columns (ask)
    cols = pd.Index(range(1, 11), name='OFRSIZ_dec')
    d_ij = (pd.crosstab(sub['BIDSIZ_dec'], sub['OFRSIZ_dec'], dropna=False) #Count how many observations fall into each (i,j) bucket
              .reindex(index=idx, columns=cols, fill_value=0)) #(crosstab gives the raw counts by bid/ask decile)
    return d_ij / d_ij.to_numpy().sum()#convert to probability

In [29]:
d_XLF = compute_dij('XLF')
d_QQQQ = compute_dij('QQQQ')
d_AAPL1 = compute_dij('AAPL1')
d_AAPL2 = compute_dij('AAPL2')
d_AAPL3 = compute_dij('AAPL3')
d_JPM = compute_dij('JPM')
d_MSFT = compute_dij('MSFT')

In [30]:
def fit_hidden_liquidity(u_ij, d_ij, bounds=(-10.0, 10.0)):
    if isinstance(u_ij, pd.DataFrame):
        u_ij = u_ij.reindex(index=range(1,11), columns=range(1,11))
    if isinstance(d_ij, pd.DataFrame):
        d_ij = d_ij.reindex(index=range(1,11), columns=range(1,11)).fillna(0)

    U = np.array(u_ij, dtype=float, copy=True) 
    D = np.array(d_ij, dtype=float, copy=True)

    if U.shape != D.shape:
        raise ValueError(f"Shape mismatch: U{U.shape} vs D{D.shape}")
    m, n = U.shape
    I = np.arange(1, m+1)[:, None]/10
    J = np.arange(1, n+1)[None, :]/10 # To be consistent with the article (as i,j are presented a decile 0.1->0.9)
    def loss(H):
        F = (I + H) / (I + J + 2.0*H)         
        R = (U - F)**2
        L = np.sum(R * D) #Objective funtion of the pipeline
        return L if np.isfinite(L) else np.inf
    res = minimize_scalar(loss, bounds=bounds, method="bounded")
    return float(res.x), float(res.fun)

In [31]:
h_XLF = fit_hidden_liquidity(p_XLF, d_XLF)[0]
h_QQQQ = fit_hidden_liquidity(p_QQQQ, d_QQQQ)[0]
h_AAPL1 = fit_hidden_liquidity(p_AAPL1, d_AAPL1)[0]
h_AAPL2 = fit_hidden_liquidity(p_AAPL2, d_AAPL2)[0]
h_AAPL3 = fit_hidden_liquidity(p_AAPL3, d_AAPL3)[0]
h_JPM = fit_hidden_liquidity(p_JPM, d_JPM)[0]
h_MSFT = fit_hidden_liquidity(p_MSFT, d_MSFT)[0]

In [32]:
h_values = {
    'XLF': h_XLF,
    'QQQQ': h_QQQQ,
    'AAPL1': h_AAPL1,
    'AAPL2': h_AAPL2,
    'AAPL3': h_AAPL3,
    'JPM': h_JPM,
    'MSFT': h_MSFT
}

h_table = pd.DataFrame(list(h_values.items()), columns=['ticker', 'implied_h'])
display(np.round(h_table,3))

,ticker,implied_h
0,XLF,0.140
1,QQQQ,0.217
2,AAPL1,1.305
3,AAPL2,0.615
4,AAPL3,0.251
5,JPM,0.191
6,MSFT,0.068


In [33]:
def compute_u_ij_model(h): #Using H
    I = np.arange(1, 11)[:, None]
    J = np.arange(1, 11)[None, :]
    Uhat = (I + h) / (I + J + 2*h)
    index = pd.Index(range(1, 11), name='bid_size_dec')
    columns = pd.Index(range(1, 11), name='ask_size_dec')
    return pd.DataFrame(Uhat, index=index, columns=columns)

In [34]:
p_hat_XLF = compute_u_ij_model(h_XLF)
p_hat_QQQQ = compute_u_ij_model(h_QQQQ)
p_hat_AAPL1 = compute_u_ij_model(h_AAPL1)
p_hat_AAPL2 = compute_u_ij_model(h_AAPL2)
p_hat_AAPL3 = compute_u_ij_model(h_AAPL3)
p_hat_JPM = compute_u_ij_model(h_JPM)
p_hat_MSFT = compute_u_ij_model(h_MSFT)

In [35]:
display(np.round(p_hat_XLF,3))

ask_size_dec,1,2,3,4,5,6,7,8,9,10
bid_size_dec,,,,,,,,,,
1,0.500,0.348,0.266,0.216,0.182,0.157,0.138,0.123,0.111,0.101
2,0.652,0.500,0.405,0.341,0.294,0.258,0.231,0.208,0.190,0.174
3,0.734,0.595,0.500,0.431,0.379,0.338,0.305,0.278,0.256,0.236
4,0.784,0.659,0.569,0.500,0.446,0.403,0.367,0.337,0.312,0.290
5,0.818,0.706,0.621,0.554,0.500,0.456,0.419,0.387,0.360,0.336
6,0.843,0.742,0.662,0.597,0.544,0.500,0.462,0.430,0.402,0.377
7,0.862,0.769,0.695,0.633,0.581,0.538,0.500,0.467,0.439,0.413
8,0.877,0.792,0.722,0.663,0.613,0.570,0.533,0.500,0.471,0.445
9,0.889,0.810,0.744,0.688,0.640,0.598,0.561,0.529,0.500,0.474


In [36]:
display(np.round(p_hat_QQQQ,3))

ask_size_dec,1,2,3,4,5,6,7,8,9,10
bid_size_dec,,,,,,,,,,
1,0.500,0.354,0.274,0.224,0.189,0.164,0.144,0.129,0.117,0.106
2,0.646,0.500,0.408,0.345,0.298,0.263,0.235,0.212,0.194,0.178
3,0.726,0.592,0.500,0.433,0.381,0.341,0.308,0.281,0.259,0.239
4,0.776,0.655,0.567,0.500,0.447,0.404,0.369,0.339,0.314,0.292
5,0.811,0.702,0.619,0.553,0.500,0.456,0.420,0.388,0.361,0.338
6,0.836,0.737,0.659,0.596,0.544,0.500,0.463,0.431,0.403,0.378
7,0.856,0.765,0.692,0.631,0.580,0.537,0.500,0.468,0.439,0.414
8,0.871,0.788,0.719,0.661,0.612,0.569,0.532,0.500,0.471,0.446
9,0.883,0.806,0.741,0.686,0.639,0.597,0.561,0.529,0.500,0.474


In [37]:
display(np.round(p_hat_AAPL1,3))

ask_size_dec,1,2,3,4,5,6,7,8,9,10
bid_size_dec,,,,,,,,,,
1,0.500,0.411,0.349,0.303,0.268,0.240,0.217,0.199,0.183,0.169
2,0.589,0.500,0.434,0.384,0.344,0.312,0.285,0.262,0.243,0.226
3,0.651,0.566,0.500,0.448,0.406,0.371,0.341,0.316,0.295,0.276
4,0.697,0.616,0.552,0.500,0.457,0.421,0.390,0.363,0.340,0.319
5,0.732,0.656,0.594,0.543,0.500,0.463,0.432,0.404,0.380,0.358
6,0.760,0.688,0.629,0.579,0.537,0.500,0.468,0.440,0.415,0.393
7,0.783,0.715,0.659,0.610,0.568,0.532,0.500,0.472,0.446,0.424
8,0.801,0.738,0.684,0.637,0.596,0.560,0.528,0.500,0.475,0.451
9,0.817,0.757,0.705,0.660,0.620,0.585,0.554,0.525,0.500,0.477


In [38]:
display(np.round(p_hat_AAPL2,3))

ask_size_dec,1,2,3,4,5,6,7,8,9,10
bid_size_dec,,,,,,,,,,
1,0.500,0.382,0.309,0.259,0.223,0.196,0.175,0.158,0.144,0.132
2,0.618,0.500,0.420,0.362,0.318,0.283,0.256,0.233,0.214,0.198
3,0.691,0.580,0.500,0.439,0.392,0.353,0.322,0.296,0.273,0.254
4,0.741,0.638,0.561,0.500,0.451,0.411,0.377,0.349,0.324,0.303
5,0.777,0.682,0.608,0.549,0.500,0.459,0.424,0.395,0.369,0.346
6,0.804,0.717,0.647,0.589,0.541,0.500,0.465,0.434,0.408,0.384
7,0.825,0.744,0.678,0.623,0.576,0.535,0.500,0.469,0.442,0.418
8,0.842,0.767,0.704,0.651,0.605,0.566,0.531,0.500,0.473,0.448
9,0.856,0.786,0.727,0.676,0.631,0.592,0.558,0.527,0.500,0.475


In [39]:
display(np.round(p_hat_AAPL3,3))

ask_size_dec,1,2,3,4,5,6,7,8,9,10
bid_size_dec,,,,,,,,,,
1,0.500,0.357,0.278,0.227,0.192,0.167,0.147,0.132,0.119,0.109
2,0.643,0.500,0.409,0.346,0.300,0.265,0.237,0.214,0.196,0.180
3,0.722,0.591,0.500,0.433,0.382,0.342,0.310,0.283,0.260,0.241
4,0.773,0.654,0.567,0.500,0.447,0.405,0.370,0.340,0.315,0.293
5,0.808,0.700,0.618,0.553,0.500,0.457,0.420,0.389,0.362,0.339
6,0.833,0.735,0.658,0.595,0.543,0.500,0.463,0.431,0.403,0.379
7,0.853,0.763,0.690,0.630,0.580,0.537,0.500,0.468,0.439,0.414
8,0.868,0.786,0.717,0.660,0.611,0.569,0.532,0.500,0.471,0.446
9,0.881,0.804,0.740,0.685,0.638,0.597,0.561,0.529,0.500,0.474


In [40]:
display(np.round(p_hat_JPM,3))

ask_size_dec,1,2,3,4,5,6,7,8,9,10
bid_size_dec,,,,,,,,,,
1,0.500,0.352,0.272,0.221,0.187,0.161,0.142,0.127,0.115,0.105
2,0.648,0.500,0.407,0.343,0.297,0.261,0.234,0.211,0.192,0.177
3,0.728,0.593,0.500,0.432,0.381,0.340,0.307,0.280,0.258,0.238
4,0.779,0.657,0.568,0.500,0.447,0.404,0.368,0.338,0.313,0.291
5,0.813,0.703,0.619,0.553,0.500,0.456,0.419,0.388,0.361,0.337
6,0.839,0.739,0.660,0.596,0.544,0.500,0.463,0.430,0.402,0.378
7,0.858,0.766,0.693,0.632,0.581,0.537,0.500,0.467,0.439,0.414
8,0.873,0.789,0.720,0.662,0.612,0.570,0.533,0.500,0.471,0.446
9,0.885,0.808,0.742,0.687,0.639,0.598,0.561,0.529,0.500,0.474


In [41]:
display(np.round(p_hat_MSFT,3))

ask_size_dec,1,2,3,4,5,6,7,8,9,10
bid_size_dec,,,,,,,,,,
1,0.500,0.341,0.258,0.208,0.174,0.150,0.131,0.117,0.105,0.096
2,0.659,0.500,0.403,0.337,0.290,0.254,0.226,0.204,0.186,0.170
3,0.742,0.597,0.500,0.430,0.377,0.336,0.303,0.276,0.253,0.234
4,0.792,0.663,0.570,0.500,0.445,0.401,0.365,0.335,0.310,0.288
5,0.826,0.710,0.623,0.555,0.500,0.455,0.418,0.386,0.359,0.335
6,0.850,0.746,0.664,0.599,0.545,0.500,0.462,0.429,0.401,0.376
7,0.869,0.774,0.697,0.635,0.582,0.538,0.500,0.467,0.438,0.412
8,0.883,0.796,0.724,0.665,0.614,0.571,0.533,0.500,0.471,0.445
9,0.895,0.814,0.747,0.690,0.641,0.599,0.562,0.529,0.500,0.474
